## First of all, we create two classes to work with, to be able to create a database, connect to it and create tables in it

In [1]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [2]:
import sqlite3
from sqlite3 import Error


def create_Database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(':memory:')
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


Now, we create the database

In [ ]:
#create database/connect
database = r"C:\sqlite\db\pythondb.db"
conn = create_Database(database)

After that, we create the main classes to work with GPT3, Example, so we can feed it with examples and then GPT3 to create the object

In [3]:
from doctest import Example

import openai
import uuid


openai.api_key = "YOUR_API_KEY"

class Example:

    # Stores an input, output pair and formats it to prime the model
    def __init__(self, input, output):
        self.input = input
        self.output = output
        self.id = uuid.uuid4().hex

    # To obtain the input provided for an example
    def get_input(self):
        return self.input

    # To obtain the output provided for an example
    def get_output(self):
        return self.output

    # To obtain the unique id of an example
    def get_id(self):
        return self.id


class GPT3:
    """
      Params engine: Model to be used. Options are Davinci, Babbage, Ada and Curie.
      temperature: Amount of randomness to be introduced in the predictions of the
      model Setting a higher value of temperature would be useful for creative
      applications whereas a lower value will be suitable for well defined answers.
      max_tokens: Maximum length of number of tokens accepted by the prompt.
   """

    # initialises parameters and adds default values
    def __init__(self, engine='davinci', temperature=0.5, max_tokens=100,

                 input_prefix="input: ", input_suffix="\n", output_prefix="output: ",
                 output_suffix="\n\n", append_output_prefix_to_query=False):
        self.examples = {}
        self.engine = engine
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.input_prefix = input_prefix
        self.input_suffix = input_suffix
        self.output_prefix = output_prefix
        self.output_suffix = output_suffix
        self.append_output_prefix_to_query = append_output_prefix_to_query
        self.stop = (output_suffix + input_prefix).strip()

    # Adds an example to the model object. Example is an instance of the Example class.
    def add_example(self, ex):
        assert isinstance(ex, Example), "Please create an Example object."
        self.examples[ex.get_id()] = ex

    # Converts all the examples to a particular format to prime the model.
    def get_prime_text(self):
        return "".join(
            [self.format_example(ex) for ex in self.examples.values()])

    # Creates a query for the API request
    def craft_query(self, prompt):
        q = self.get_prime_text(
        ) + self.input_prefix + prompt + self.input_suffix

        if self.append_output_prefix_to_query:
            q = q + self.output_prefix
        return q

    # Calls the API using the Completion endpoint with the specified values of the parameters
    def submit_request(self, prompt):
        response = openai.Completion.create(engine=self.engine,
                                            prompt=self.craft_query(prompt),
                                            max_tokens=self.max_tokens,
                                            temperature=self.temperature,
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop=self.stop)
        return response

    # Formats the input output pair with appropriate prefixes and suffixes
    def format_example(self, ex):
        return self.input_prefix + ex.get_input(
        ) + self.input_suffix + self.output_prefix + ex.get_output(
        ) + self.output_suffix


And after that, we create some methods that will be useful to work with the database. 
FindIdAttr that finds the id of the attribute on the table we want to
FindNextId, so we can add examples without repeating any Id (Primary Key) and it gives no error
NumberOfAttributes that works for the insert data, that gives us the number of attr we will add
TableColumns, that gives the name of the attributes on the table, for inserting data as well
And then the last three methods that are just to convert the results given by the database commands into strings and readable outputs

In [ ]:
def FindIdAttr(table, attr, conn):
    cur = conn.cursor()
    cur.execute("""SELECT """ + table + """Id 
    FROM """ + table + """
    WHERE """ + table + """Name = " """ + attr + """ " """)

    rows = cur.fetchall()
    st = convertToReadable(convertTuple(rows))
    return st

def FindNextId(table, conn):
    try:
        command = ("""SELECT * FROM """ + table + """
                  ORDER BY """ + table + """Id DESC
                    """)
        cur = conn.cursor()
        cur.execute(command)
        rows = cur.fetchall()
        st = convertToReadable(convertTuple(rows))
        nextId = int(st[1]) + 1
    except:
        nextId = 0

    return nextId

def numberOfAttributes(rows):
    sqlAux = '('
    for i in range(len(rows)):
        sqlAux += '?,'
    sqlAux = sqlAux[:-1]
    sqlAux += ')'
    return sqlAux

def tableColumns(tableName):
    if conn is not None:
        cur = conn.cursor()

        cur.execute("SELECT name FROM pragma_table_info('" + tableName + "')")
        rows = cur.fetchall()
        str = convertTuple(rows)
        str = convertToReadable(str)
        # str store the columns name in the table

    else:
        print("Error! cannot create the database connection.")
    return str

def convertTuple(tup):
    st = ''.join(map(str, tup))
    return st


def convertToReadable(string):
    str = string.replace("',)('", ",")
    str = str.replace("('", "")
    str = str.replace("',)", "")
    return str

def convertIdToReadable(string):
    str = string.replace(",)", "")
    str = str.replace("(", "")
    return str

Now we create the GPT3 object and the connection to de database

In [ ]:
gpt3 = GPT3(engine="davinci", temperature=0.5, max_tokens=400)

database = r"C:\sqlite\db\pythonsqlite.db"

# create a database connection
conn = create_connection(database)

cur = conn.cursor()

Now, before we start working with the database, let's try some things with GPT3 trying with SELECT statements

In [ ]:
gpt3.add_example(Example('Fetch unique values of DEPARTMENT from Worker table.','Select distinct DEPARTMENT from Worker;'))

gpt3.add_example(Example("Get all details of workers who have top 5 salaries.","Select * from Worker where SALARY in (Select distinct top 5 SALARY from Worker order by SALARY desc"))

gpt3.add_example(Example("Display the highest salary from the Worker table.","Select max(Salary) from Worker;"))

gpt3.add_example(Example("Fetch the count of employees working in the department Admin.","SELECT COUNT(*) FROM worker WHERE DEPARTMENT = 'Admin';"))

gpt3.add_example(Example("Get all details of the Workers whose SALARY lies between 5000 and 10000.","Select * from Worker where SALARY between 5000 and 10000;"))

gpt3.add_example(Example("Fetch the count of employees working in the department Admin.","SELECT COUNT(*) FROM worker WHERE DEPARTMENT = 'Admin';"))

gpt3.add_example(Example('Print the first three characters of FIRST_NAME from Worker table.','Select substring(FIRST_NAME,1,3) from Worker;'))

prompt = """Get count of workers whose salary is over 1000"""
response = gpt3.submit_request(prompt)
response.choices[0].text  # This chooses the topmost response from multiple outputs if any.
print(response)

In [ ]:
gpt3.add_example(Example("""select all available columns from users table""", """SELECT id, signup_dt, email, plan_type FROM users"""))

gpt3.add_example((Example("""select all available columns from charges table""", """SELECT amount, user_id, and charge_dt FROM charges""")))

gpt3.add_example((Example("""what was the largest payment amount made in the past 30 days?""", """SELECT MAX(amount) FROM charges WHERE charge_dt > NOW() - INTERVAL '30 days'""")))

gpt3.add_example((Example("""when did jane@seekwell.io make her first payment?""" , """SELECT MIN(charge_dt) as last_payment_dt from charges 
LEFT JOIN
users ON users.id = charges.user_id
WHERE users.email = 'jane@seekwell.io'""")))

gpt3.add_example((Example("""how many new users signed up in each of the last 2 months?""" , """SELECT sum(case when signup_dt>= now() - interval '1 month' then 1 else 0 end) as signups_this_month,
sum(case when signup_dt>= now() - interval '2 months' and signup_dt < now() - interval '1 month'  then 1 else 0 end) as signups_last_month
FROM users""")))

gpt3.add_example((Example("""what percent of users have an email ending in 'gmail.com'?""", """SELECT SUM(CASE WHEN email like '%gmail.com' then 1 else 0 end)/COUNT(*) as percent_gmail
FROM users""")))

prompt = """what percent of users who signed up converted to paid?"""

response = gpt3.submit_request(prompt)
response.choices[0].text  # This chooses the topmost response from multiple outputs if any.

print(response)

As we can see, just adding some simple examples, it works so good, even though it is not perfect, still so good with just few examples

Now, as we know how easy is for GPT3 to work with select statements, lets try with create tables, and adding them to our database

In [ ]:
gpt3.add_example(Example("""Create table with all the album's attributes""", """
CREATE TABLE Album (
  `AlbumId` INT NOT NULL,
  `AlbumName` VARCHAR(45) NOT NULL,
  `AlbumDate` DATE NOT NULL,
  PRIMARY KEY (`AlbumId`)
)
"""))

gpt3.add_example((Example("""Create table with all the artist's attributes""", """
CREATE TABLE Artist (
  `ArtistId` INT NOT NULL,
  `ArtistName` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`ArtistId`)
)
""")))

gpt3.add_example((Example("""Create table with all the worker's attributes""", """
CREATE TABLE Worker (
  `WrokerId` INT NOT NULL,
  `WorkerName` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`WrokerId`)
)
""")))

gpt3.add_example((Example("""Create table with all the worker's attributes AND add the company id from the company table""" , """
CREATE TABLE Customer ( 
  `WrokerId` INT NOT NULL,
  `WorkerName` VARCHAR(45) NOT NULL,
  `CompanyId` INT NOT NULL,
  PRIMARY KEY (`WrokerId`, `CompanyId`))
FOREIGN KEY (`CompanyId`) REFERENCES Company(`CompanyId`)
)
""")))

gpt3.add_example((Example("""Create table with all the album's attributes AND add the artist id from the artist table""" , """
CREATE TABLE Album (
  `AlbumId` INT NOT NULL,
  `AlbumName` VARCHAR(45) NOT NULL,
  `ArtistId` INT NOT NULL,
  PRIMARY KEY (`AlbumId`, `ArtistId`),
FOREIGN KEY (`ArtistId`) REFERENCES Company(`ArtistId`)
)
""")))

prompt = """Create table with all the buy's data AND add the customer id from the customer table"""
response = gpt3.submit_request(prompt)
response.choices[0].text  # This chooses the topmost response from multiple outputs if any.
response.replace("""
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": output: 
""", "")
response.replace(""""
    }
  ],
  "created": 1639490607,
  "id": "cmpl-4F2gpP6b441AwpqatVdL3B07HSWvZ",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
""", "")

response.replace("\n", """
""")

x = response.replace("output:", "")

if conn is not None:
    # create table
    Proj.SQL.create_table(conn, x)

else:
    print("Error! cannot create the database connection.")
    
print(x)

Result for Worker Table

In [ ]:
CREATE TABLE Worker (
  `WorkerId` INT NOT NULL,
  `WorkerName` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`WorkerId`)
)

Result for Customer Table

In [ ]:
CREATE TABLE Customer (
  `CustomerId` INT NOT NULL,
  `CustomerName` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`CustomerId`)
)

Result for Buy Table

In [ ]:
CREATE TABLE Buy (
  `BuyId` INT NOT NULL,
  `BuyName` VARCHAR(45) NOT NULL,
#This will be better as BuyPrice, but anyways an useful result
  `CustomerId` INT NOT NULL,
  `WorkerId` INT NOT NULL,
  PRIMARY KEY (`BuyId`, `CustomerId`, `WorkerId`),
FOREIGN KEY (`CustomerId`) REFERENCES Customer(`CustomerId`)
FOREIGN KEY (`WorkerId`) REFERENCES Worker(`WorkerId`)
)

Done as well with Artist and Album as in the examples to use them later

Now, we saw that still with foreign keys, it works very good creating tables with few examples

We are going to see some examples of how sqlite3 works so we have a better understanding of the next coding cells

In [ ]:
#no need for execute, example of adding data (after execute, it ends up in the same state than it started)
sql = ''' INSERT INTO Worker(WrokerId,WorkerName)
                  VALUES(?,?) '''
task = (1, 'Antonio')
cur = conn.cursor()
cur.execute(sql, task)
#example of Select
cur.execute("SELECT * FROM Worker")
rows = cur.fetchall()
print(convertToReadable(convertTuple(rows)))
#example of delete
cur.execute("DELETE FROM Worker")
#View Table columns
cur.execute("SELECT name FROM pragma_table_info('Worker')")
rows = cur.fetchall()
print(convertToReadable(convertTuple(rows)))

Also we add some auxiliar data to work with it later

In [ ]:
table = "Artist"
cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))

sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)

task = (FindNextId(table, conn), "Los Del Rio") #Id 0
cur.execute(sql, task)
conn.commit()

task = (FindNextId(table, conn), "Eminem")#Id 1
cur.execute(sql, task)
conn.commit()

task = (FindNextId(table, conn), "Estopa")#Id 2
cur.execute(sql, task)
conn.commit()

We try some examples of adding data without using sqlite code, just with natural language

In [ ]:
gpt3.add_example(Example("""Add to albums table, where we store Id, Name and Date, the album Fiesta Macarena published on 1996-06-02""","""INSERT INTO `album`(AlbumId,AlbumName,AlbumDate) VALUES (1,Fiesta Macarena, 1996-06-02)"""))

gpt3.add_example(Example("""Add to artists table, where we store Id and Name, the artist David Bisbal""","""INSERT INTO `artist`(ArtistId,ArtistName) VALUES (1,David Bisbal)"""))

gpt3.add_example(Example("""Add to workers table, where we store Id, Name and CompanyId, the customer Miguel Avila, who works in Samsung""","""INSERT INTO `worker`(WrokerId,WorkerName,CompanyId) VALUES (1,Miguel Avila, 1)"""))

prompt = """Add to customers buys table, where we store Id, Date, Price and CustomerId a buy taht costed 1200 made on 2021-12-12 by customer 1 """
response = gpt3.submit_request(prompt)
response.choices[0].text  # This chooses the topmost response from multiple outputs if any.
print(response)


Finally, as we know how to extract data of databases with sqlite and add examples, lets use all of this with GPT3 and see if it is able to add data with just a phrase 

In [ ]:
gpt3.add_example(Example("""Add to the Artist table, David Bisbal""", """
table = "Artist"
cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))

sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)

task = (FindNextId(table, conn), "David Bisbal")
cur.execute(sql, task)
conn.commit()
"""))

gpt3.add_example(Example("""Add to the Album table, Fiesta Macarena published by artist Los Del Rio""", """
table = "Album"
table2 = "Artist"

cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))

sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)

cur = conn.cursor()
cur.execute('''SELECT ''' + table2 + '''Id 
FROM ''' + table2 + '''
WHERE ''' + table2 + '''Name = "Los Del Rio" ''')

rows = cur.fetchall()
artistId = convertIdToReadable(convertTuple(rows))
                 
task = (FindNextId(table, conn), "Fiesta Macarena", int(artistId))

cur.execute(sql, task)
conn.commit()
"""))

gpt3.add_example(Example("""Add to the Album table, The Eminem Show published by artist Eminem""", """
table = "Album"
table2 = "Artist"

cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))

sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)

cur = conn.cursor()
cur.execute('''SELECT ''' + table + '''Id 
FROM ''' + table + '''
WHERE ''' + table + '''Name = "Eminem" ''')

rows = cur.fetchall()
artistId = convertIdToReadable(convertTuple(rows))
                 
task = (FindNextId(table, conn), "The Eminem Show", int(artistId))

cur.execute(sql, task)
conn.commit()
"""))

prompt = "Add to the Buy table, FirstBuy done by customer David Klotz and worker Antonio Jesus"
response = gpt3.submit_request(prompt)
response.choices[0].text  # This chooses the topmost response from multiple outputs if any.

response.replace("""
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": output: 
""", "")
response.replace(""""
    }
  ],
  "created": 1639490607,
  "id": "cmpl-4F2gpP6b441AwpqatVdL3B07HSWvZ",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
""", "")

response.replace("\n", """
""")

x = response.replace("output:", "")

print(response)

This is the result we got for Worker, a code that adds data with no problem

In [ ]:
table = "Worker"
cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))
sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)
cur = conn.cursor()
task = (FindNextId(table, conn), "Antonio J")
cur.execute(sql, task)
conn.commit()
cur.execute("SELECT * FROM Worker")
print(cur.fetchall())

This is the result we got for Customer, a code that adds data with no problem

In [ ]:
table = "Customer"
cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))
sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)
task = (FindNextId(table, conn), "David Klotz")
cur.execute(sql, task)
conn.commit()
cur.execute("SELECT * FROM Customer")
print(cur.fetchall())

This is the result we got for Buy, a code that doesn't works perfectly but still, just one small thing of easy repair missing

In [ ]:
table = "Buy"
table2 = "Customer"
table3 = "Worker"
cur.execute("SELECT name FROM pragma_table_info('" + table + "')")
rows = cur.fetchall()
st = convertToReadable(convertTuple(rows))
sql = '''INSERT INTO `''' + table + '''`(''' + st + ''')
VALUES ''' + numberOfAttributes(rows)
cur = conn.cursor()
cur.execute('''SELECT ''' + table2 + '''Id
FROM ''' + table2 + '''
WHERE ''' + table2 + '''Name = "David Klotz" ''')
rows = cur.fetchall()
customerId = convertIdToReadable(convertTuple(rows))
cur.execute('''SELECT WrokerId
FROM ''' + table3 + '''
WHERE ''' + table3 + '''Name = "Antonio Jesus" ''')
rows = cur.fetchall()
workerId = convertIdToReadable(convertTuple(rows))
task = (FindNextId(table, conn), customerId, workerId) #BuyNameMissing
#task = (FindNextId(table, conn), "FirstBuy", customerId, workerId); This will be the one working
cur.execute(sql, task)
conn.commit()

After all of this research, we can say that working with GPT3 to create SQL code is an easy task that only needs to be fed with few examples. If we fed it with just two more examples in each section, probably it will work perfectly and also much faster than any of us coding.